<a href="https://colab.research.google.com/github/wordbrew/SVD-Requirements/blob/main/11_22_23_Stable_diffusion_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Connect Google Drive
from google.colab import drive

print("Connecting...")
drive.mount('/content/drive')

In [ ]:
#@markdown # Clone the Repo
import os

#Create a new subfolder in Google Drive and change directory
path = '/content/drive/MyDrive/SVD'
os.makedirs(path, exist_ok=True)  # Create the folder if it doesn't exist
os.chdir(path)  # Change the current working directory to the new folder

# Set the destination path where the repository is supposed to be cloned
generative_models_path = '/content/drive/MyDrive/SVD/generative-models'

# Check if the destination path already exists
if not os.path.exists(generative_models_path):
    # If the path does not exist, clone the repository
    !git clone https://github.com/Stability-AI/generative-models.git {generative_models_path}
else:
    print(f"Directory '{generative_models_path}' already exists, skipping clone.")

# Change directory to the 'generative-models' directory
os.chdir(generative_models_path)

In [ ]:
#@markdown # Install Dependencies
from IPython.display import clear_output

# Install dependencies
!pip install -r requirements/pt2.txt
!pip install .

clear_output()
print("Dependencies successfully installed.")

In [ ]:
import os
import requests
from tqdm import tqdm

#@markdown # Model Download/Load
SVD_Model_Version = "SVD" #@param [ "SVD", "SVD_XT"]

# Set the destination path
destination_path = '/content/drive/MyDrive/SVD/generative-models'

# Check if 'checkpoint' folder exists, and create it if not
checkpoint_dir = os.path.join(destination_path, 'checkpoints')
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    print("Created directory: ", checkpoint_dir)
else:
    print("Directory already exists: ", checkpoint_dir)

# URLs for the files to be downloaded
model_url = "https://huggingface.co/stabilityai/stable-video-diffusion-img2vid/resolve/main/svd.safetensors"
decoder_url = "https://huggingface.co/stabilityai/stable-video-diffusion-img2vid/resolve/main/svd_image_decoder.safetensors"
model_xt_url = "https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt.safetensors"
decoder_xt_url = "https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt/resolve/main/svd_xt_image_decoder.safetensors"

# Function to download a file from a URL with a progress bar
def download_file(url, folder):
    # Get the filename from the URL
    filename = url.split('/')[-1]
    filepath = os.path.join(folder, filename)

    # Check if the file already exists
    if os.path.exists(filepath):
        print(f"File already exists: {filename}")
        return

    # Send a GET request to the URL
    response = requests.get(url, stream=True)

    # Get the total file size from headers
    total_size = int(response.headers.get('content-length', 0))

    # Write the file to the specified path with a progress bar
    if response.status_code == 200:
        with open(filepath, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(chunk_size=1024):
                size = f.write(data)
                bar.update(size)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {filename}")

# Download the files based on model version
def SVD():
    download_file(model_url, checkpoint_dir)
    download_file(decoder_url, checkpoint_dir)

def SVD_XT():
    download_file(model_xt_url, checkpoint_dir)
    download_file(decoder_xt_url, checkpoint_dir)

# Invoke the appropriate function based on the selected model version
if SVD_Model_Version == "SVD":
    SVD()
elif SVD_Model_Version == "SVD_XT":
    SVD_XT()

In [ ]:
#@markdown # Setup ngrok API Key

#@markdown If you don't already have an account, get one at https://dashboard.ngrok.com/

from IPython.display import clear_output

#Install pyngrok
!pip install pyngrok

#Type API Key in field to the right
api_key = "" #@param {type:"string"}

#Set API Key
if api_key.strip():
    # Set API Key
    !ngrok config add-authtoken {api_key}
    clear_output()
    print("ngrok setup.")
else:
    print("Please enter an API key. If you don't have one, get it at https://dashboard.ngrok.com/")

In [ ]:
#@markdown # Run the Program with UI
import os
from pyngrok import ngrok

!cp '/content/drive/MyDrive/SVD/generative-models/scripts/demo/video_sampling.py' '/content/drive/MyDrive/SVD/generative-models/'

#set PythonPath
os.environ['PYTHONPATH'] = "/content/drive/MyDrive/SVD/generative-models/"

# Define the path to the directory
directory_path = "/content/drive/MyDrive/SVD/generative-models/"

# Change the current working directory to the specified path
os.chdir(directory_path)

# Check current working directory to verify if the change was successful
print("Current Working Directory: ", os.getcwd())

#Set NGROK url
public_url = ngrok.connect("8501").public_url
print("Public URL", public_url)

# Run video_sampling.py via streamlit
!streamlit run video_sampling.py --server.port 8501